<a href="https://colab.research.google.com/github/JaminUbuntu/JaminUbuntu/blob/main/Logistic_Regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install pyspark py4j

In [114]:
from pyspark.sql import SparkSession

In [115]:
spark = SparkSession.builder.appName("pyspark_test").getOrCreate()

In [116]:
df = spark.read.csv("/content/live.csv", header=True, inferSchema=True)

In [117]:
df.printSchema()

root
 |-- num_reactions: double (nullable = true)
 |-- num_comments: double (nullable = true)
 |-- num_shares: double (nullable = true)
 |-- num_likes: double (nullable = true)
 |-- num_loves: double (nullable = true)
 |-- num_wows: double (nullable = true)
 |-- num_hahas: double (nullable = true)
 |-- num_sads: double (nullable = true)
 |-- num_angrys: double (nullable = true)
 |-- status_type_link: double (nullable = true)
 |-- status_type_photo: double (nullable = true)
 |-- status_type_status: double (nullable = true)
 |-- status_type_video: double (nullable = true)



In [118]:
df.describe().show()

+-------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+-------------------+
|summary|      num_reactions|        num_comments|          num_shares|           num_likes|           num_loves|            num_wows|           num_hahas|            num_sads|          num_angrys|    status_type_link| status_type_photo| status_type_status|  status_type_video|
+-------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+-------------------+
|  count|               7050|                7050|                7050|                7050|                7050|                7050|                7050|           

In [119]:
df.columns

['num_reactions',
 'num_comments',
 'num_shares',
 'num_likes',
 'num_loves',
 'num_wows',
 'num_hahas',
 'num_sads',
 'num_angrys',
 'status_type_link',
 'status_type_photo',
 'status_type_status',
 'status_type_video']

In [120]:
from pyspark.ml.feature import VectorAssembler

In [123]:
 assembler = VectorAssembler(inputCols=[
'num_reactions',
 'num_comments',
 'num_shares',
 'num_likes',
 'num_loves',
 'num_wows',
 'num_hahas',
 'num_sads',
 'num_angrys',
 'status_type_link',
 'status_type_photo',
 'status_type_status',
 'status_type_video'],
  outputCol='features')

In [124]:
output = assembler.transform(df)

In [129]:
df_final = output.select('features', 'status_type_video')

In [130]:
df_final.show()

+--------------------+-----------------+
|            features|status_type_video|
+--------------------+-----------------+
|[0.11231422505307...|              1.0|
|(13,[0,3,10],[0.0...|              0.0|
|[0.04819532908704...|              1.0|
|(13,[0,3,10],[0.0...|              0.0|
|(13,[0,3,4,10],[0...|              0.0|
|(13,[0,1,3,4,5,10...|              0.0|
|[0.10679405520169...|              1.0|
|[0.06263269639065...|              1.0|
|(13,[0,1,3,4,10],...|              0.0|
|(13,[0,1,2,3,4,10...|              0.0|
|(13,[0,1,2,3,4,5,...|              0.0|
|(13,[0,1,3,4,5,10...|              0.0|
|(13,[0,1,2,3,4,5,...|              0.0|
|(13,[0,1,3,4,5,10...|              0.0|
|(13,[0,1,3,4,5,10...|              0.0|
|[0.07048832271762...|              1.0|
|(13,[0,1,2,3,4,12...|              1.0|
|[0.03184713375796...|              1.0|
|(13,[0,1,2,3,4,6,...|              1.0|
|(13,[0,1,3,4,10],...|              0.0|
+--------------------+-----------------+
only showing top

In [131]:
train_data, test_data = df_final.randomSplit([0.7, 0.3])

In [148]:
train_data.describe().show()

+-------+------------------+
|summary| status_type_video|
+-------+------------------+
|  count|              4989|
|   mean|0.3315293646021247|
| stddev|0.4708100202523569|
|    min|               0.0|
|    max|               1.0|
+-------+------------------+



In [149]:
test_data.describe().show

<bound method DataFrame.show of DataFrame[summary: string, status_type_video: string]>

In [132]:
from pyspark.ml.classification import LogisticRegression

In [133]:
lr = LogisticRegression(featuresCol='features', labelCol='status_type_video')

In [134]:
lrModel = lr.fit(train_data)

In [136]:
lrModel

LogisticRegressionModel: uid=LogisticRegression_7aabaadfa551, numClasses=2, numFeatures=13

In [137]:
lrModel.summary

In [135]:
lrModelSummary = lrModel.summary

In [138]:
lrModelSummary.predictions.show()

+--------------------+-----------------+--------------------+--------------------+----------+
|            features|status_type_video|       rawPrediction|         probability|prediction|
+--------------------+-----------------+--------------------+--------------------+----------+
|(13,[0,1,2,3,4,5,...|              0.0|[18.5611378840036...|[0.99999999131039...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1688319224113...|[0.99999999526759...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1768821940297...|[0.99999999530553...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1718363762945...|[0.99999999528179...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1527066425845...|[0.99999999519066...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1876805404585...|[0.99999999535595...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1736641261717...|[0.99999999529040...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1700628440090...

In [139]:
lrModelSummary.predictions.describe().show()

+-------+------------------+------------------+
|summary| status_type_video|        prediction|
+-------+------------------+------------------+
|  count|              4989|              4989|
|   mean|0.3315293646021247|0.3315293646021247|
| stddev|0.4708100202523569|0.4708100202523569|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [140]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [141]:
pred_labels = lrModel.evaluate(test_data)

In [142]:
pred_labels.predictions.show()

+--------------------+-----------------+--------------------+--------------------+----------+
|            features|status_type_video|       rawPrediction|         probability|prediction|
+--------------------+-----------------+--------------------+--------------------+----------+
|(13,[0,1,2,3,4,5,...|              0.0|[19.1019211636231...|[0.99999999494011...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1256065902346...|[0.99999999505854...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1651503807125...|[0.99999999525013...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1752245540562...|[0.99999999529774...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1522523576100...|[0.99999999518847...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1301360381755...|[0.99999999508087...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.0646282867760...|[0.99999999474784...|       0.0|
|(13,[0,1,2,3,4,5,...|              0.0|[19.1781933710965...

In [143]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='status_type_video')

In [144]:
auc = eval.evaluate(pred_labels.predictions)

In [145]:
auc

1.0